# Testing NOBIL data

### connecting with duckdb

In [130]:
import duckdb
from pathlib import Path


path_duckdb = Path.cwd().parent / "data_warehouse"

with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.csmd_table_dump;")
    df_csmd = cursor.df()


df_csmd.head()


,id,name,ocpidb_mapping_stasjon_id,street,house_number,zipcode,city,municipality_id,municipality,county_id,...,created,updated,station_status,land_code,international_id,_dlt_load_id,_dlt_id,description_of_location,user_comment,contact_info
0,14671,Agnesfridsvägen 193,SE#VAT#75dbcba6-8bdf-4c70-9b5e-c7e2ab4dd50d,Agnesfridsvägen,193,21375,Malmö,1280,Malmö,12,...,2023-04-24 11:31:42+02:00,2025-09-21 01:13:17+02:00,1,SWE,SWE_14671,1765794546.3968723,W7Z70oeFL/dK3w,None,None,None
1,54376,Reload Filipstad,SE#VIR#SEVIRLJX9T0WRY,John Ericssonsgatan,8,68230,Filipstad,1782,Filipstad,17,...,2025-06-13 03:44:32+02:00,2025-06-14 03:44:28+02:00,1,SWE,SWE_54376,1765794546.3968723,5s2dmDY6IJpnqA,None,None,None
2,36573,Källhage Fast AB,SE#CNE#10e955c3-9683-4e7a-de1f-08dc39d02872,Toredalsgatan,3,41871,Göteborg,1480,Göteborg,14,...,2024-11-02 00:00:34+01:00,2025-11-16 20:18:09+01:00,1,SWE,SWE_36573,1765794546.3968723,AHTJ/g6BK4mQdw,None,None,None
3,54401,Jönköpings kommun - P-hus Smedjan,SE#VLT#6cbb6162-2630-40ef-a66d-44a91bf9a6ed,Södra Strandgatan,59,55320,Jönköping,0680,Jönköping,06,...,2025-06-13 07:00:56+02:00,2025-06-13 07:00:56+02:00,1,SWE,SWE_54401,1765794546.3968723,/h051P+jnFh0Eg,None,None,None
4,18856,500,DK#MON#s384017,Flygarvägen,37,17569,Järfälla,0123,Järfälla,01,...,2023-12-15 12:57:07+01:00,2025-11-18 05:22:17+01:00,1,SWE,SWE_18856,1765794546.3968723,KC194rk0Z49FNg,None,None,None


In [ ]:
for kommun in df_csmd["municipality"].unique():
    print(kommun)

In [131]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.status_online_table_dump;")
    df_st = cursor.df()
    
df_st.head()

,station_id,updated_date,attr_key,attrtypeid,attrname,attrvalid,trans,attrval,_dlt_load_id,_dlt_id
0,14671,2025-09-21 01:13:17+02:00,24,24,Open 24h,1,Yes,1,1765794546.3968723,3CKnsG93QiYBjQ
1,14671,2025-09-21 01:13:17+02:00,21,21,Real-time information,2,No,,1765794546.3968723,pn3ocTJ6mHpyGg
2,14671,2025-09-21 01:13:17+02:00,22,22,Public funding,4,None,,1765794546.3968723,bsTh6UvaSWdYCA
3,14671,2025-09-21 01:13:17+02:00,7,7,Parking fee,2,No,False,1765794546.3968723,WhlFMZ4/TLBYOA
4,14671,2025-09-21 01:13:17+02:00,6,6,Time limit,2,No,0,1765794546.3968723,uNOuJKH2Bww0hw


In [ ]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.connector_table_dump;")
    df_conn = cursor.df()
    
df_conn

In [ ]:
from datetime import date
import datetime

x = datetime.datetime.now()

today_date = date(x.year, x.month, x.day).isoformat()

print(today_date)


In [ ]:
import requests 
from constants.utils import API_KEY_NOBIL


params = {
    "apikey": API_KEY_NOBIL,
    "countrycode": "SWE",
    "format": "json",
    "file": "false",
}

url = "https://nobil.no/api/server/datadump.php"

r = requests.get(url=url, params=params)
r.raise_for_status()

In [ ]:
data = r.json()

In [ ]:
data["chargerstations"][0][""]

In [ ]:
from pathlib import Path
import pandas as pd
path = Path.cwd().parents[0]/"data"



df = pd.read_excel(path/"trafikanalys.xlsx", sheet_name= "Data")





In [ ]:
df.head()

In [ ]:

df = df.reset_index().rename({"index": "id"}, axis=1)
df["id"] = df["id"].apply(lambda x: x+1)




In [ ]:
df 

In [ ]:

df = df.to_dict(orient= "records")



In [ ]:
df

# Trafikanalys filter kommun och län


In [ ]:
import duckdb
from pathlib import Path


path_duckdb = Path.cwd().parent / "data_warehouse"

with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    
    df= conn.execute("select * from staging.trafikanalys;")
    
    df_trafikanalys = df.df()
    
    

In [ ]:

# ~df_trafikanalys, "~" betyder Not, så det som inte är lika med det i contains kommer det att printas ut
# astype för sätta data typ, str vektoresering för att använda strip på varje post, str med contains är vektoresera varje post och använder
# contains med regex, case är boolesk om det ska vara case sensitive och regex boolesk för tolka det man skickar in är regular expression. 
filter = df_trafikanalys[~df_trafikanalys["kommun"].astype(str).str.strip().str.contains(r"Okänd kommun|Totalt", case= False, regex= True, na= False)]



In [ ]:

kommuner = sorted(filter["kommun"].dropna().unique().tolist())






In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)


print(kommuner)

In [ ]:

# Samma som för kommun fast nu är det för län
df_filter_län = df_trafikanalys[~df_trafikanalys["l_n"].astype(str).str.strip().str.contains(r"Okänt län|Totalt",case= False, regex= True, na=False)]





In [ ]:
df_filter_län

In [ ]:
län = sorted(df_filter_län["l_n"].dropna().unique().tolist())

In [ ]:
print(län)

# Create csv file in seed for kommun and län

In [ ]:



df_trafikanalys = df_trafikanalys[~df_trafikanalys["kommun"].astype(str).str.strip().str.contains(r"Okänd kommun|Totalt", case= False, regex= True, na= False)]
df_trafikanalys = df_trafikanalys[~df_trafikanalys["l_n"].astype(str).str.strip().str.contains(r"Okänt län|Totalt",case= False, regex= True, na=False)]



In [ ]:

# Create csv file in seeds
path = Path().cwd().parent/"data_transform/seeds"

df_kommun_lan = (df_trafikanalys[["kommun", "l_n"]].drop_duplicates())

df_kommun_lan.to_csv(path/"municipality_lan.csv", index=False, encoding= "utf-8")


In [ ]:
len(df_kommun_lan["l_n"].unique())

# Exploration marts

In [3]:
import duckdb
from pathlib import Path


PATH_DUCKDB = Path.cwd().parent / "data_warehouse" / "data.duckdb"
    
with duckdb.connect(PATH_DUCKDB, read_only=True) as conn:
    
    query = 'SELECT * FROM marts.nr_charger;'

    curser = conn.execute(query=query)
    
    df = curser.fetchdf()
    
    df

In [ ]:
len(df)

289

In [21]:
df

,antal_ladd_stationer,antal_operator,laddpunkter,procent_antal_snabb_laddpunkter,municipality,county
0,14,2,81.0,7.14,emmaboda,kalmar
1,94,33,869.0,28.72,helsingborg,skåne
2,61,20,468.0,19.67,kungsbacka,halland
3,16,12,66.0,43.75,sunne,värmland
4,46,15,172.0,21.74,västervik,kalmar
...,...,...,...,...,...,...
284,10,4,31.0,10.00,bengtsfors,västra götaland
285,6,3,22.0,83.33,sorsele,västerbotten
286,6,5,29.0,83.33,hammarö,värmland
287,6,5,38.0,16.67,tjörn,västra götaland


In [19]:
county = df.groupby("county")["antal_ladd_stationer"].sum()

In [20]:
county.loc["stockholm"]

np.int64(2185)

In [23]:
import duckdb
from pathlib import Path


PATH_DUCKDB = Path.cwd().parent / "data_warehouse" / "data.duckdb"
    
with duckdb.connect(PATH_DUCKDB, read_only=True) as conn:
    
    query = 'SELECT * FROM marts.infrastructur;'

    curser = conn.execute(query=query)
    
    df_infrastructur = curser.fetchdf()
    
df_infrastructur

,antal_ladd_stationer,total_vehicle,total_kw,municipality,county
0,45,9720.0,2204.0,falköping,västra götaland
1,116,14746.0,3973.4,gotland,gotland
2,102,44653.0,7299.8,örebro,örebro
3,34,18051.0,2454.0,falun,dalarna
4,38,56776.0,3868.0,sollentuna,stockholm
...,...,...,...,...,...
284,22,12708.0,3038.0,piteå,norrbotten
285,12,2493.0,1449.6,gällivare,norrbotten
286,4,659.0,22.0,ljusnarsberg,örebro
287,7,433.0,1600.0,arvidsjaur,norrbotten


## Graph

In [39]:
import plotly_express as px
from backend.data_processing import query_analytics


df = query_analytics("nr_charger")

data = df[df["county"] == "stockholm"].sort_values(
    by="antal_ladd_stationer", ascending=True
)

fig = px.bar(
    data,
    x="antal_ladd_stationer",
    y="municipality",
    title="Antal laddstationer för varje kommun",
    orientation="h",
    template="simple_white",
    width=500,
)

fig.update_layout(height=35 * len(data) + 150,margin= dict(l=0, r=0, t=40, b=0),showlegend = False)
fig.update_yaxes(ticks="", minor=dict(ticks=""))
fig.show()

# Graph with snowflake

## connect with database

In [1]:
import os 
from pathlib import Path
from dotenv import load_dotenv
import snowflake.connector
import pandas as pd



    
load_dotenv()

with snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE") )as conn:
    
    query = 'SELECT * FROM marts.nr_charger;'

        
    cur = conn.cursor()   
    data =cur.execute(query)
    
    df = data.fetch_pandas_all()
    
df 

,ANTAL_LADD_STATIONER,ANTAL_OPERATOR,LADDPUNKTER,ANTAL_SNABB_LADD_STATIONER,MUNICIPALITY,COUNTY
0,444,51,6436,56,göteborg,västra götaland
1,37,15,219,6,oskarshamn,kalmar
2,15,10,90,6,munkedal,västra götaland
3,32,13,161,7,vara,västra götaland
4,28,6,68,2,askersund,örebro
...,...,...,...,...,...,...
284,4,4,11,3,övertorneå,norrbotten
285,14,10,37,3,flen,södermanland
286,3,1,29,1,skinnskatteberg,västmanland
287,16,7,73,2,håbo,uppsala


In [25]:
import os 
from dotenv import load_dotenv
import snowflake.connector
import pandas as pd



    
load_dotenv()

with snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE") )as conn:
    
    query = 'SELECT * FROM marts.nr_charger;'

        
    cur = conn.cursor()   
    data =cur.execute(query)
    
    df_infra = data.fetch_pandas_all()
    
df_infra

,ANTAL_LADD_STATIONER,ANTAL_OPERATOR,LADDPUNKTER,ANTAL_SNABB_LADD_STATIONER,TOTAL_VEHICLE,MUNICIPALITY,COUNTY
0,91,25,598,18,12703,jönköping,jönköping
1,11,9,43,5,1057,sala,västmanland
2,15,12,87,4,3391,motala,östergötland
3,56,19,858,14,9016,mölndal,västra götaland
4,98,25,849,25,13062,linköping,östergötland
...,...,...,...,...,...,...,...
284,5,3,30,1,416,hagfors,värmland
285,11,1,22,0,645,svenljunga,västra götaland
286,9,5,45,2,712,tomelilla,skåne
287,18,9,203,2,4160,upplands väsby,stockholm


In [45]:

df_infra.query("COUNTY == 'stockholm'")





stockholm_df = (
    df_infra.query("COUNTY == 'stockholm'")[["MUNICIPALITY", "LADDPUNKTER", "TOTAL_VEHICLE", "ANTAL_SNABB_LADD_STATIONER", "ANTAL_LADD_STATIONER"]]
    )

stockholm_df["ELBIL_PER_LADDPUNKT"] = stockholm_df["TOTAL_VEHICLE"]/ stockholm_df["LADDPUNKTER"]
stockholm_df["PROCENT_SNABB"] = round(stockholm_df["ANTAL_SNABB_LADD_STATIONER"] / stockholm_df["ANTAL_LADD_STATIONER"] * 100, 2)


stockholm_df

 


,MUNICIPALITY,LADDPUNKTER,TOTAL_VEHICLE,ANTAL_SNABB_LADD_STATIONER,ANTAL_LADD_STATIONER,ELBIL_PER_LADDPUNKT,PROCENT_SNABB
11,solna,1091,18822,9,81,17.252062,11.11
14,botkyrka,135,6064,7,21,44.918519,33.33
17,norrtälje,242,4694,8,55,19.396694,14.55
36,stockholm,10830,143490,74,1242,13.249307,5.96
38,upplands-bro,147,2354,7,53,16.013605,13.21
78,haninge,155,6435,8,36,41.516129,22.22
92,ekerö,121,2798,3,18,23.123967,16.67
110,huddinge,404,8668,14,75,21.455446,18.67
114,tyresö,92,3860,6,24,41.956522,25.00
115,värmdö,211,4693,7,41,22.241706,17.07


In [313]:
import plotly_express as px



ticksval_x = [1,2,5,10,20,50,100,200,500,1000,2000,5000,10000]
ticksval_y = [10, 20, 50, 100]

stockholm_df["PROCENT_SNABB_SIZE"] = stockholm_df["PROCENT_SNABB"].clip(lower= 1)

fig = px.scatter(
    stockholm_df,
    x=stockholm_df["LADDPUNKTER"],
    y=stockholm_df["ELBIL_PER_LADDPUNKT"],
    hover_name=stockholm_df["MUNICIPALITY"],
    size= stockholm_df["PROCENT_SNABB_SIZE"],
    color= stockholm_df["MUNICIPALITY"],
    size_max=25,
    opacity=0.6,
    template="simple_white"
)



# Add line where elbil per laddpunkt is 10. 
fig.add_hline(y=10, line_width= 2, line_color= "#02010F", opacity=0.6)


fig.update_traces(marker=dict(line=dict(width=0)))

# update y- and x-axel and its ticks
fig.update_xaxes(type="log", tickmode= "array",  tickvals= ticksval_x, ticktext= [str(v) for v in ticksval_x], ticks= "", title_text= "")
fig.update_xaxes(showline= True, linewidth= 1, linecolor="rgba(0,0,0,0.3)")
fig.update_yaxes(type= "log", tickmode= "array", tickvals= ticksval_y, ticktext= [str(v) for v in ticksval_y], ticks= "", title_text= "")
fig.update_yaxes(showline= True, linewidth=1, linecolor="rgba(0,0,0,0.3)")


# Give titel to y-axel
fig.add_annotation(
    text= "ELBILAR PER LADDPUNKT",
    font= dict(
        size= 13,
        color= "black",
        family= "Ariel"
    ),
    xref= "paper",
    yref= "paper",
    x= -0.0605,
    y= 1.10,
    xanchor="center",
    yanchor= "top",
    opacity= 1,
    showarrow=False
)

# Give title to x-axel
fig.add_annotation(
    text= "LADDPUNKTER PER KOMMUN",
    font= dict(
        size= 15,
        color="black",
        family= "Ariel"
        ), 
    xref= "paper",
    yref= "paper",
    x= 0.1,
    y= -0.18,
    opacity=1,
    xanchor="center",
    yanchor= "bottom",
    showarrow=False
)


# update title 
fig.update_layout(
    title={
        "text": "Elbil per laddpunkt.<br>" 
        "Cirkelns storlek bestäms över hur stor procent kommuner snabb laddare.",
        "y": 0.95,
        "x": 0.11,
        "xanchor": "left",
        "yanchor": "top"
        
    },
    margin= dict(l= 155, b= 70, t= 100)
)



fig.show()